# TP : Comparaison des modèles LSTM et RNN pour la classification de critiques de films Allociné

## Contexte :

En traitement automatique du langage naturel (NLP), les modèles récurrents comme les RNN (Réseaux de Neurones Récurrents) et les LSTM (Long Short Term Memory) sont couramment utilisés pour les tâches de classification de texte.

Dans ce TP, vous allez comparer la performance de ces deux types de modèles sur une tâche de classification de critiques de films du site Allociné.

Les modèles seront entraînés à prédire si une critique est positive ou négative en fonction du texte de la critique.

## Objectifs :

1. **Préparation des données :**
    - Utilisez la librairie `datasets` pour charger le jeu de données `allocine`.
    - Effectuez le prétraitement nécessaire sur les critiques (par exemple, la tokenisation).

2. **Entraînement des modèles :**
    - Implémentez un modèle RNN et un modèle LSTM en utilisant `PyTorch`.
    - Entraînez les deux modèles sur vos données d'entraînement.

3. **Évaluation des modèles :**
    - Évaluez la performance de vos modèles sur vos données de test.
    - Comparez la performance des deux modèles. Quel modèle a le mieux performé ? Pouvez-vous expliquer pourquoi ?

4. **Interprétation des résultats:** Quels sont les aspects du modèle qui pourraient être améliorés?

## Consignes :

Vous êtes libres de choisir l'architecture exacte de vos modèles (nombre de couches, taille des couches, etc.), mais gardez à l'esprit que l'objectif est de comparer un RNN et un LSTM.
  
## Ressources :

- [PyTorch](https://pytorch.org/)
- [Librairie `datasets`](https://huggingface.co/docs/datasets/)

## Bonne chance !

# Code de base

In [ ]:
from datasets import load_dataset

dataset = load_dataset("allocine")
dataset

In [ ]:
train_dataset, valid_dataset, test_dataset = dataset["train"], dataset["validation"], dataset["test"]

In [ ]:
train_dataset['review'][0], train_dataset['label'][0]

In [ ]:
# Get the spacy model for the tokenizer
!python -m spacy download fr_core_news_sm

Let's see how to creat a simple tokenizer with vocabulary

In [ ]:
from torchtext.data.utils import get_tokenizer
from torchtext.vocab import build_vocab_from_iterator

tokenizer = get_tokenizer('spacy', language="fr")


def yield_tokens(data_iter):
    for text in data_iter:
        yield tokenizer(text.lower())


texts = ["Bonjour je vais très bien", "Le lisp et trop cool !"]

vocab = build_vocab_from_iterator(yield_tokens(texts), specials=["<unk>", "<pad>", "<s>", "</s>"])
vocab.set_default_index(vocab["<unk>"])

print(f"Vocabulary: {vocab.get_itos()}")

In [ ]:
# To test hour vocabulary

sentence = "TOTO je vais très bien le lisp TOTO"

# Tokenize the sentence
tokens = tokenizer(sentence)

# Convert tokens to indices via the vocab
encoded_sentence = [vocab[token] for token in tokens]

print(f"Encoded sentence: {encoded_sentence}")

## LSTM & RNN USAGE

Voici un code simple pour utilisez un RNN et LSTM

### RNN

In [ ]:
import random
import torch
import torch.nn as nn

vocab_size = 5
sequence_length = 10
embedding_dim = 8
num_layers = 2
hidden_size = 20
# On crée des tokens aléatoire pour la demo (int entre 0 et vocab_size)
random_tokens = [random.randint(0, vocab_size) for _ in range(sequence_length)]
# ON crée notre Embedding num_embeddings et le nombres de mots dans notre vocabulaire
embedding = nn.Embedding(num_embeddings=vocab_size + 1, embedding_dim=embedding_dim)

# Instantiate an RNN with input size equal to embedding_dim, hidden state size 20, and 2 layers.
rnn = nn.RNN(input_size=embedding_dim,
             hidden_size=hidden_size,
             num_layers=num_layers,
             # [batch, seq len, features]
             batch_first=True)

# Initialize the hidden state.
# [num_layers, batch size, hidden_size]
h_0 = torch.randn(num_layers, 1, hidden_size)

# Forward propagate the RNN with input and initial hidden state.
embeddings = embedding(torch.tensor([random_tokens]))
print("embeddings", embeddings.size())
output, hn = rnn(embeddings, h_0)
print("Output: ", output.shape)
print("hn: ", hn.shape)

### LSTM

In [ ]:
vocab_size = 5
sequence_length = 10
embedding_dim = 8
num_layers = 2
hidden_size = 20
random_tokens = [random.randint(0, vocab_size) for _ in range(sequence_length)]
embedding = nn.Embedding(num_embeddings=vocab_size + 1, embedding_dim=embedding_dim)

# Instantiate an LSTM with input size equal to embedding_dim, hidden state size 20, and 2 layers.
lstm = nn.LSTM(input_size=embedding_dim,
               hidden_size=hidden_size,
               num_layers=num_layers,
               # [batch, seq len, features]
               batch_first=True)

# Initialize the hidden state and cell state.
# [num_layers, batch size, hidden_size]
h_0 = torch.randn(num_layers, 1, hidden_size)
c_0 = torch.randn(num_layers, 1, hidden_size)

# Forward propagate the RNN with input and initial hidden state.
embeddings = embedding(torch.tensor([random_tokens]))
print("embeddings", embeddings.size())
output, (hn, cn) = lstm(embeddings, (h_0, c_0))
print("Output: ", output.shape)
print("hn: ", hn.shape)
print("cn: ", cn.shape)